# Module 4: Geometric Transformations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/04_Module.ipynb)

**Week 7-8: Translation, Rotation, Scaling, Affine & Perspective Transformations**

## Learning Objectives
- Understand geometric transformation theory
- Apply affine transformations (rotation, scaling, translation, shear)
- Work with homogeneous coordinates
- Implement perspective transformations
- Perform image warping and remapping

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Transformation Theory

### Topics Covered
1. **Homogeneous Coordinates**: Unified representation for all transformations
2. **Affine Transformations**: Translation, rotation, scaling, shear
3. **Transformation Matrices**: 2D and 3D representations
4. **Matrix Composition**: Combining multiple transformations
5. **Perspective Transformations**: Projective geometry
6. **Interpolation Methods**: Image resampling theory

### 1. Homogeneous Coordinates

#### Why Homogeneous Coordinates?

In **standard Cartesian coordinates**, different transformations require different operations:
- Rotation: Matrix multiplication
- Scaling: Matrix multiplication
- Translation: Vector addition (not matrix multiplication!)

**Problem**: Cannot represent all transformations as matrix multiplication.

**Solution**: Use **homogeneous coordinates** - add an extra dimension!

#### Definition

A 2D point $(x, y)$ in **homogeneous coordinates** becomes:

$$
\mathbf{p} = \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
$$

Or more generally:

$$
\mathbf{p} = \begin{bmatrix} wx \\ wy \\ w \end{bmatrix}, \quad w \neq 0
$$

**Conversion back to Cartesian**:

$$
\begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} wx/w \\ wy/w \end{bmatrix}
$$

#### Key Benefits

1. **Unified representation**: All transformations are matrix multiplication
2. **Easy composition**: Multiply transformation matrices
3. **Handles infinity**: Points at infinity have $w = 0$
4. **Perspective division**: Natural handling of perspective projection

#### Translation in Homogeneous Coordinates

**Standard coordinates** (addition):
$$
\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t_x \\ t_y \end{bmatrix}
$$

**Homogeneous coordinates** (matrix multiplication!):
$$
\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = 
\begin{bmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
$$

In [ ]:
# Demonstrate homogeneous coordinates
print("=" * 70)
print("HOMOGENEOUS COORDINATES: DEMONSTRATION")
print("=" * 70)

# Define a point in Cartesian coordinates
point_cartesian = np.array([3, 4])
print(f"\nCartesian coordinates: {point_cartesian}")

# Convert to homogeneous coordinates
point_homogeneous = np.array([3, 4, 1])
print(f"Homogeneous coordinates: {point_homogeneous}")

# Different homogeneous representations (same point)
point_h2 = np.array([6, 8, 2])  # 2x scale
point_h3 = np.array([9, 12, 3])  # 3x scale

print(f"\nAlternative homogeneous representations:")
print(f"  [3, 4, 1]   → ({point_h2[0]/point_h2[2]}, {point_h2[1]/point_h2[2]})")
print(f"  [6, 8, 2]   → ({point_h2[0]/point_h2[2]}, {point_h2[1]/point_h2[2]})")
print(f"  [9, 12, 3]  → ({point_h3[0]/point_h3[2]}, {point_h3[1]/point_h3[2]})")
print(f"\nAll represent the same point: ({point_cartesian[0]}, {point_cartesian[1]})")

# Translation using homogeneous coordinates
print("\n" + "="*70)
print("TRANSLATION VIA MATRIX MULTIPLICATION")
print("="*70)

# Translation vector
t_x, t_y = 5, 3

# Translation matrix
T = np.array([
    [1, 0, t_x],
    [0, 1, t_y],
    [0, 0, 1]
])

print(f"\nTranslation matrix T (translate by ({t_x}, {t_y})):")
print(T)

# Apply translation
point_translated_h = T @ point_homogeneous
point_translated = point_translated_h[:2] / point_translated_h[2]

print(f"\nOriginal point: {point_cartesian}")
print(f"After translation: {point_translated}")
print(f"Expected: {point_cartesian + np.array([t_x, t_y])}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))

# Original point
ax.scatter(point_cartesian[0], point_cartesian[1], s=200, c='blue', 
           marker='o', label='Original', zorder=5)
ax.annotate(f'  ({point_cartesian[0]}, {point_cartesian[1]})', 
            (point_cartesian[0], point_cartesian[1]),
            fontsize=12, color='blue')

# Translated point
ax.scatter(point_translated[0], point_translated[1], s=200, c='red',
           marker='s', label='Translated', zorder=5)
ax.annotate(f'  ({point_translated[0]:.0f}, {point_translated[1]:.0f})',
            (point_translated[0], point_translated[1]),
            fontsize=12, color='red')

# Arrow showing translation
ax.arrow(point_cartesian[0], point_cartesian[1],
         t_x, t_y, head_width=0.3, head_length=0.2,
         fc='green', ec='green', linewidth=2, alpha=0.7,
         label=f'Translation ({t_x}, {t_y})')

ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.legend(fontsize=12)
ax.set_title('Translation Using Homogeneous Coordinates', fontsize=14)
ax.set_xlabel('X')
ax.set_ylabel('Y')

plt.tight_layout()
plt.show()

print("\nKey Insight: Homogeneous coordinates allow ALL transformations")
print("to be represented as matrix multiplication!")

### 2. Affine Transformations

An **affine transformation** preserves:
- Points lying on a line remain on a line (collinearity)
- Parallel lines remain parallel
- Ratios of distances along lines

#### General Form

In 2D homogeneous coordinates:

$$
\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} =
\begin{bmatrix}
a_{11} & a_{12} & t_x \\
a_{21} & a_{22} & t_y \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
$$

Or in block form:

$$
\mathbf{p}' = \begin{bmatrix} A & \mathbf{t} \\ \mathbf{0}^T & 1 \end{bmatrix} \mathbf{p}
$$

Where:
- $A \in \mathbb{R}^{2 \times 2}$: Linear transformation (rotation, scaling, shear)
- $\mathbf{t} \in \mathbb{R}^2$: Translation vector

#### Basic Affine Transformations

**1. Translation**:
$$
T(t_x, t_y) = \begin{bmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{bmatrix}
$$

**2. Scaling**:
$$
S(s_x, s_y) = \begin{bmatrix}
s_x & 0 & 0 \\
0 & s_y & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

**3. Rotation** (counterclockwise by angle $\theta$):
$$
R(\theta) = \begin{bmatrix}
\cos\theta & -\sin\theta & 0 \\
\sin\theta & \cos\theta & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

**4. Shear** (horizontal):
$$
H_x(m) = \begin{bmatrix}
1 & m & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

**5. Reflection** (across y-axis):
$$
F_y = \begin{bmatrix}
-1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
# Load sample image
!wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg -O cat.jpg
img = cv2.imread('cat.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]

print(f"Image shape: {img.shape}")

plt.figure(figsize=(8, 6))
plt.imshow(img_rgb)
plt.title('Original Image')
plt.axis('off')
plt.show()

## 4.1 Translation

**Mathematical definition**:
$$
\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t_x \\ t_y \end{bmatrix}
$$

OpenCV uses a **2×3 affine matrix** (projection of 3×3 homogeneous matrix):
$$
M = \begin{bmatrix} 1 & 0 & t_x \\ 0 & 1 & t_y \end{bmatrix}
$$

In [ ]:
# Translation matrix
t_x, t_y = 100, 50
M_translate = np.float32([
    [1, 0, t_x],
    [0, 1, t_y]
])

print("Translation matrix M (2×3):")
print(M_translate)
print(f"\nTranslation: ({t_x}, {t_y})")

# Apply translation
img_translated = cv2.warpAffine(img, M_translate, (w, h))

# Display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_translated, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Translated by ({t_x}, {t_y})')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 4.2 Rotation

**Rotation around origin** by angle $\theta$:
$$
\begin{bmatrix} x' \\ y' \end{bmatrix} =
\begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix}
$$

**Rotation around arbitrary center** $(c_x, c_y)$:
1. Translate center to origin: $T(-c_x, -c_y)$
2. Rotate: $R(\theta)$
3. Translate back: $T(c_x, c_y)$

**Combined matrix**:
$$
M = T(c_x, c_y) \cdot R(\theta) \cdot T(-c_x, -c_y)
$$

In [ ]:
# Rotation parameters
center = (w // 2, h // 2)
angle = 45  # degrees
scale = 1.0

# Get rotation matrix using OpenCV
M_rotate = cv2.getRotationMatrix2D(center, angle, scale)

print(f"Rotation matrix (2×3):")
print(M_rotate)
print(f"\nCenter: {center}")
print(f"Angle: {angle}°")
print(f"Scale: {scale}")

# Manual construction
theta = np.radians(angle)
cos_theta = np.cos(theta)
sin_theta = np.sin(theta)

print(f"\nManual calculation:")
print(f"  cos({angle}°) = {cos_theta:.6f}")
print(f"  sin({angle}°) = {sin_theta:.6f}")

# Apply rotation
img_rotated = cv2.warpAffine(img, M_rotate, (w, h))

# Display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].plot(center[0], center[1], 'r+', markersize=15, markeredgewidth=2)
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_rotated, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Rotated {angle}° around center')
axes[1].plot(center[0], center[1], 'r+', markersize=15, markeredgewidth=2)
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 4.3 Scaling

**Mathematical definition**:
$$
\begin{bmatrix} x' \\ y' \end{bmatrix} =
\begin{bmatrix}
s_x & 0 \\
0 & s_y
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix}
$$

- $s_x, s_y > 1$: Enlargement
- $0 < s_x, s_y < 1$: Reduction
- $s_x = s_y$: Uniform scaling
- $s_x \neq s_y$: Non-uniform scaling

In [ ]:
# Scaling factors
scale_x, scale_y = 1.5, 1.5

# Method 1: Using cv2.resize()
new_w = int(w * scale_x)
new_h = int(h * scale_y)
img_scaled_resize = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

# Method 2: Using warpAffine
M_scale = np.float32([
    [scale_x, 0, 0],
    [0, scale_y, 0]
])
img_scaled_warp = cv2.warpAffine(img, M_scale, (new_w, new_h))

print(f"Original size: {w} × {h}")
print(f"Scale factors: ({scale_x}, {scale_y})")
print(f"New size: {new_w} × {new_h}")
print(f"\nScaling matrix M:")
print(M_scale)

# Display
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title(f'Original\n{w}×{h}')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_scaled_resize, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Scaled (cv2.resize)\n{new_w}×{new_h}')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_scaled_warp, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Scaled (warpAffine)\n{new_w}×{new_h}')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 4.4 Transformation Composition

**Order matters!** Matrix multiplication is not commutative.

**Example**: Rotate then translate vs Translate then rotate

$$
M_1 = T \cdot R \quad \text{(rotate around origin, then translate)}
$$

$$
M_2 = R \cdot T \quad \text{(translate, then rotate around origin)}
$$

$$
M_1 \neq M_2
$$

**Common operation**: Rotate around center with scaling
$$
M = T(c_x, c_y) \cdot S(s) \cdot R(\theta) \cdot T(-c_x, -c_y)
$$

In [ ]:
# Demonstrate order dependency
print("=" * 70)
print("TRANSFORMATION ORDER MATTERS!")
print("=" * 70)

# Create a small test shape
test_img = np.zeros((300, 300, 3), dtype=np.uint8)
pts = np.array([[50, 100], [150, 100], [100, 50]], dtype=np.int32)
cv2.fillPoly(test_img, [pts], (0, 255, 0))

# Transformation parameters
angle = 30
tx, ty = 100, 50

# Method 1: Rotate then Translate
M_rot = cv2.getRotationMatrix2D((50, 50), angle, 1.0)
M_trans = np.float32([[1, 0, tx], [0, 1, ty]])

# Compose: T * R (apply R first, then T)
# Need to convert to 3x3 for multiplication
M_rot_3x3 = np.vstack([M_rot, [0, 0, 1]])
M_trans_3x3 = np.vstack([M_trans, [0, 0, 1]])

M_1 = M_trans_3x3 @ M_rot_3x3
M_1_2x3 = M_1[:2, :]

result1 = cv2.warpAffine(test_img, M_1_2x3, (300, 300))

# Method 2: Translate then Rotate
M_2 = M_rot_3x3 @ M_trans_3x3
M_2_2x3 = M_2[:2, :]

result2 = cv2.warpAffine(test_img, M_2_2x3, (300, 300))

print(f"\nTransformation 1: Rotate {angle}°, then translate ({tx}, {ty})")
print(f"Matrix M1 = T × R:")
print(M_1_2x3)

print(f"\nTransformation 2: Translate ({tx}, {ty}), then rotate {angle}°")
print(f"Matrix M2 = R × T:")
print(M_2_2x3)

print(f"\nMatrices are different: {not np.allclose(M_1_2x3, M_2_2x3)}")

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(result1, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Rotate → Translate\n(T × R)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(result2, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Translate → Rotate\n(R × T)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Transformation order changes the result!")

## 4.5 Perspective Transformation

**Perspective transformations** (projective transformations) are more general than affine.

#### Mathematical Definition

Full 3×3 homogeneous transformation:

$$
\begin{bmatrix} wx' \\ wy' \\ w \end{bmatrix} =
\begin{bmatrix}
h_{11} & h_{12} & h_{13} \\
h_{21} & h_{22} & h_{23} \\
h_{31} & h_{32} & h_{33}
\end{bmatrix}
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
$$

**Perspective division** to get Cartesian coordinates:

$$
x' = \frac{h_{11}x + h_{12}y + h_{13}}{h_{31}x + h_{32}y + h_{33}}
$$

$$
y' = \frac{h_{21}x + h_{22}y + h_{23}}{h_{31}x + h_{32}y + h_{33}}
$$

#### Properties

- **Preserves**: Lines remain lines
- **Does NOT preserve**: Parallelism, angles, distances
- **8 degrees of freedom**: 9 parameters, but scale-invariant
- **Requires**: 4 point correspondences to solve

In [ ]:
# Perspective transformation example: Document scanner
print("=" * 70)
print("PERSPECTIVE TRANSFORMATION: DOCUMENT RECTIFICATION")
print("=" * 70)

# Create a tilted rectangle image
doc_img = np.ones((400, 600, 3), dtype=np.uint8) * 255
cv2.rectangle(doc_img, (50, 50), (550, 350), (200, 200, 200), -1)
cv2.putText(doc_img, 'DOCUMENT', (200, 220), cv2.FONT_HERSHEY_SIMPLEX,
            2, (0, 0, 0), 3)

# Source points (quadrilateral - perspective view)
src_pts = np.float32([
    [100, 150],  # Top-left
    [500, 100],  # Top-right
    [50, 350],   # Bottom-left
    [550, 300]   # Bottom-right
])

# Destination points (rectangle - frontal view)
dst_pts = np.float32([
    [0, 0],      # Top-left
    [400, 0],    # Top-right
    [0, 300],    # Bottom-left
    [400, 300]   # Bottom-right
])

# Calculate perspective transformation matrix
M_perspective = cv2.getPerspectiveTransform(src_pts, dst_pts)

print("\nPerspective transformation matrix H (3×3):")
print(M_perspective)
print(f"\nSource points (quadrilateral):")
print(src_pts)
print(f"\nDestination points (rectangle):")
print(dst_pts)

# Apply perspective transformation
rectified = cv2.warpPerspective(doc_img, M_perspective, (400, 300))

# Visualize source quadrilateral
doc_img_viz = doc_img.copy()
cv2.polylines(doc_img_viz, [src_pts.astype(np.int32)], True, (0, 0, 255), 3)
for i, pt in enumerate(src_pts):
    cv2.circle(doc_img_viz, tuple(pt.astype(int)), 8, (255, 0, 0), -1)
    cv2.putText(doc_img_viz, str(i), tuple(pt.astype(int) + 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

# Display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(cv2.cvtColor(doc_img_viz, cv2.COLOR_BGR2RGB))
axes[0].set_title('Perspective View\n(4 source points marked)')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(rectified, cv2.COLOR_BGR2RGB))
axes[1].set_title('Rectified (Frontal View)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Application: Document scanning, camera calibration, AR")

## 4.6 Interpolation Methods

When transforming images, output pixels map to **non-integer** input coordinates.

**Interpolation** estimates pixel values at fractional positions.

### Nearest Neighbor
$$
f(x, y) = f(\text{round}(x), \text{round}(y))
$$

- Fastest
- Produces blocky results

### Bilinear Interpolation

Linear interpolation in both directions:

$$
f(x, y) = (1-a)(1-b)f(x_0, y_0) + a(1-b)f(x_1, y_0) + (1-a)bf(x_0, y_1) + abf(x_1, y_1)
$$

Where:
- $x_0 = \lfloor x \rfloor$, $x_1 = \lceil x \rceil$
- $y_0 = \lfloor y \rfloor$, $y_1 = \lceil y \rceil$
- $a = x - x_0$, $b = y - y_0$

### Bicubic Interpolation

Uses 4×4 neighborhood with cubic polynomials:

$$
f(x, y) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
$$

- Smoother than bilinear
- Slower computation

In [ ]:
# Compare interpolation methods
print("=" * 70)
print("INTERPOLATION METHODS COMPARISON")
print("=" * 70)

# Scale up by 3x to see differences
scale_factor = 3
new_size = (w * scale_factor, h * scale_factor)

# Different interpolation methods
img_nearest = cv2.resize(img, new_size, interpolation=cv2.INTER_NEAREST)
img_linear = cv2.resize(img, new_size, interpolation=cv2.INTER_LINEAR)
img_cubic = cv2.resize(img, new_size, interpolation=cv2.INTER_CUBIC)
img_lanczos = cv2.resize(img, new_size, interpolation=cv2.INTER_LANCZOS4)

print(f"\nOriginal size: {w} × {h}")
print(f"Scaled size: {new_size[0]} × {new_size[1]} ({scale_factor}x)")
print(f"\nInterpolation methods:")
print(f"  INTER_NEAREST:  Fastest, blocky")
print(f"  INTER_LINEAR:   Bilinear, good balance")
print(f"  INTER_CUBIC:    Bicubic, smoother")
print(f"  INTER_LANCZOS4: Highest quality, slowest")

# Display
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

# Show small crop to see details
crop = slice(200, 400), slice(200, 400)

axes[0, 0].imshow(cv2.cvtColor(img_nearest, cv2.COLOR_BGR2RGB)[crop])
axes[0, 0].set_title('INTER_NEAREST\n(Blocky)', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img_linear, cv2.COLOR_BGR2RGB)[crop])
axes[0, 1].set_title('INTER_LINEAR\n(Bilinear)', fontsize=12)
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(img_cubic, cv2.COLOR_BGR2RGB)[crop])
axes[1, 0].set_title('INTER_CUBIC\n(Bicubic)', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_lanczos, cv2.COLOR_BGR2RGB)[crop])
axes[1, 1].set_title('INTER_LANCZOS4\n(Highest Quality)', fontsize=12)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Better interpolation = smoother but slower!")

## Summary

### Key Concepts

1. **Homogeneous Coordinates**: Unified matrix representation for all transformations
2. **Affine Transformations**: Preserve collinearity and parallelism
3. **Perspective Transformations**: Most general linear transformation of 2D space
4. **Transformation Composition**: Order matters!
5. **Interpolation**: Critical for image quality

### Transformation Hierarchy

$$
\text{Euclidean} \subset \text{Similarity} \subset \text{Affine} \subset \text{Perspective}
$$

| Type | DOF | Preserves | Matrix Form |
|------|-----|-----------|-------------|
| Translation | 2 | Distances, angles, parallelism | $\begin{bmatrix}1 & 0 & t_x\\0 & 1 & t_y\\0 & 0 & 1\end{bmatrix}$ |
| Euclidean | 3 | Distances, angles | Rotation + Translation |
| Similarity | 4 | Angles, parallelism | Uniform scale + Rotation + Translation |
| Affine | 6 | Parallelism, ratios | $\begin{bmatrix}a & b & c\\d & e & f\\0 & 0 & 1\end{bmatrix}$ |
| Perspective | 8 | Only lines | $\begin{bmatrix}a & b & c\\d & e & f\\g & h & 1\end{bmatrix}$ |

### OpenCV Functions Reference

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Translation | `cv2.warpAffine(img, M, size)` | M = 2×3 translation matrix |
| Rotation | `cv2.getRotationMatrix2D(center, angle, scale)` | Returns 2×3 matrix |
| Scaling | `cv2.resize(img, size, interpolation)` | INTER_NEAREST/LINEAR/CUBIC |
| Affine | `cv2.getAffineTransform(src, dst)` | 3 point pairs → 2×3 matrix |
| Perspective | `cv2.getPerspectiveTransform(src, dst)` | 4 point pairs → 3×3 matrix |
| Apply Affine | `cv2.warpAffine(img, M, size)` | M is 2×3 |
| Apply Perspective | `cv2.warpPerspective(img, M, size)` | M is 3×3 |

**Next**: Module 5 - Edge Detection